In [1]:
# Parameters cell (must be tagged as "parameters" in the notebook)
execution_date = None
dag_run_id = None
ds = None
ds_nodash = None

In [2]:
# Parameters
execution_date = "2025-04-23T02:56:25.099096+00:00"
dag_run_id = "manual__2025-04-23T02:56:25.099096+00:00"
ds = "2025-04-23"
ds_nodash = "20250423"


In [3]:
!pip install kagglehub pandas psycopg2-binary sqlalchemy

In [4]:
import kagglehub
print(dir(kagglehub))

['KaggleDatasetAdapter', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'auth', 'cache', 'clients', 'colab_cache_resolver', 'competition', 'competition_download', 'config', 'dataset_download', 'dataset_load', 'dataset_upload', 'datasets', 'datasets_enums', 'datasets_helpers', 'env', 'exceptions', 'gcs_upload', 'get_package_asset_path', 'handle', 'http_resolver', 'integrity', 'kaggle_cache_resolver', 'kagglehub', 'load_dataset', 'logger', 'login', 'model_download', 'model_upload', 'models', 'models_helpers', 'notebook_output_download', 'notebooks', 'package_import', 'packages', 'registry', 'resolver', 'signing', 'tracker', 'utility_script_install', 'utility_scripts', 'whoami']


/home/megin_mathew/airflow_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [6]:
import kaggle
print("Kaggle module is successfully installed!")

Kaggle module is successfully installed!


In [7]:
kaggle.api.authenticate()
print("Kaggle API authentication successful!")

Kaggle API authentication successful!


In [8]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import os
import pandas as pd


# Define dataset folder (custom cache location)
dataset_path = "/home/megin_mathew/fashion_dataset"

# Set KaggleHub cache override
os.environ["KAGGLEHUB_CACHE"] = dataset_path

# File names in the dataset
file_names = [
    "transactions.csv", "customers.csv", "discounts.csv",
    "employees.csv", "products.csv", "stores.csv"
]

# Ensure cache directory exists
os.makedirs(dataset_path, exist_ok=True)

# Download each file individually
for file_name in file_names:
    df = kagglehub.dataset_load(
        KaggleDatasetAdapter.PANDAS,
        "ricgomes/global-fashion-retail-stores-dataset",
        file_name
    )
    
    # Save to custom cache location
    df.to_csv(os.path.join(dataset_path, file_name), index=False)
    print(f"Downloaded: {file_name} → Stored in: {dataset_path}")


Downloaded: transactions.csv → Stored in: /home/megin_mathew/fashion_dataset


/home/megin_mathew/airflow_venv/lib/python3.12/site-packages/kagglehub/pandas_datasets.py:91: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


Downloaded: customers.csv → Stored in: /home/megin_mathew/fashion_dataset


Downloaded: discounts.csv → Stored in: /home/megin_mathew/fashion_dataset


Downloaded: employees.csv → Stored in: /home/megin_mathew/fashion_dataset


Downloaded: products.csv → Stored in: /home/megin_mathew/fashion_dataset


Downloaded: stores.csv → Stored in: /home/megin_mathew/fashion_dataset


In [9]:
import os

# Set custom cache location
os.environ["KAGGLEHUB_CACHE"] = "/home/megin_mathew/fashion_dataset"

Optimized And Cleansed and uses Hadoop along with Spark

In [10]:
import os
import logging
import sys 
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, DateTime, Boolean, ForeignKey, Text, text
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
from sqlalchemy.exc import SQLAlchemyError
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, to_date, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, BooleanType

# Configure logging
log_directory = '/home/megin_mathew/logs/' # Or another directory where the user has permissions
log_file_path = os.path.join(log_directory, 'data_loader.log')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(log_file_path, encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Initialize Spark with Hadoop workaround for Windows
def create_spark_session():
    """Create and configure Spark session"""
    try:
        # Set Hadoop home directory if on Windows
        if os.name == 'nt':
            os.environ['HADOOP_HOME'] = 'C:\\hadoop'
            os.environ['PATH'] = os.environ['PATH'] + ';' + os.environ['HADOOP_HOME'] + '\\bin'
            
            # Set Python path explicitly
            python_path = sys.executable
            os.environ['PYSPARK_PYTHON'] = python_path
            os.environ['PYSPARK_DRIVER_PYTHON'] = python_path
        
        # Create temp directory if it doesn't exist
        temp_dir = "/home/megin_mathew/spark_temp"
        if not os.path.exists(temp_dir):
            os.makedirs(temp_dir)
            
        return SparkSession.builder \
            .appName("GlobalFashionRetailDataLoader") \
            .config("spark.local.dir", temp_dir) \
            .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
            .config("spark.executor.memory", "8g") \
            .config("spark.driver.memory", "8g") \
            .config("spark.memory.offHeap.enabled", "false") \
            .config("spark.sql.shuffle.partitions", "200") \
            .config("spark.default.parallelism", "200") \
            .config("spark.sql.adaptive.enabled", "true") \
            .config("spark.network.timeout", "600s") \
            .config("spark.executor.heartbeatInterval", "60s") \
            .config("spark.python.profile", "false") \
            .config("spark.executor.instances", "4") \
            .config("spark.executor.cores", "2") \
            .config("spark.driver.maxResultSize", "4g") \
            .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
            .config("spark.sql.execution.arrow.pyspark.fallback.enabled", "true") \
            .getOrCreate()
    except Exception as e:
        logger.error(f"Failed to create Spark session: {str(e)}")
        raise
# Initialize Spark
spark = create_spark_session()

# Load environment variables
load_dotenv()

# Database schema definition
Base = declarative_base()
SCHEMA_NAME = "GFRetail"

class Stores(Base):
    __tablename__ = 'stores'
    __table_args__ = {'schema': SCHEMA_NAME}
    store_id = Column(Integer, primary_key=True)
    country = Column(String(100), nullable=True)
    city = Column(String(100), nullable=True)
    store_name = Column(String(100))
    number_of_employees = Column(Integer)
    zip_code = Column(String(20))
    latitude = Column(Float, nullable=True)
    longitude = Column(Float, nullable=True)

class Employees(Base):
    __tablename__ = 'employees'
    __table_args__ = {'schema': SCHEMA_NAME}
    employee_id = Column(Integer, primary_key=True)
    store_id = Column(Integer, ForeignKey(f'{SCHEMA_NAME}.stores.store_id'))
    name = Column(String(255), nullable=True)
    position = Column(String(100), nullable=True)

class Customers(Base):
    __tablename__ = 'customers'
    __table_args__ = {'schema': SCHEMA_NAME}
    customer_id = Column(Integer, primary_key=True)
    name = Column(String(255), nullable=True)
    email = Column(String(255), nullable=True)
    join_date = Column(Date, nullable=True)
    telephone = Column(String(50), nullable=True)
    city = Column(String(100), nullable=True)
    country = Column(String(100), nullable=True)
    gender = Column(String(20), nullable=True)
    date_of_birth = Column(Date, nullable=True)
    job_title = Column(String(100), nullable=True)

class Products(Base):
    __tablename__ = 'products'
    __table_args__ = {'schema': SCHEMA_NAME}
    product_id = Column(Integer, primary_key=True)
    category = Column(String(100), nullable=True)
    sub_category = Column(String(100))
    description_pt = Column(Text)
    description_de = Column(Text)
    description_fr = Column(Text)
    description_es = Column(Text)
    description_en = Column(Text)
    description_zh = Column(Text)
    color = Column(String(50), nullable=True)
    sizes = Column(String(100), nullable=True)
    production_cost = Column(Float)

class Discounts(Base):
    __tablename__ = 'discounts'
    __table_args__ = {'schema': SCHEMA_NAME}
    id = Column(Integer, primary_key=True, autoincrement=True)
    start_date = Column(Date)
    end_date = Column(Date)
    discount_rate = Column(Float)
    description = Column(String(255), nullable=True)
    category = Column(String(100), nullable=True)
    sub_category = Column(String(100))

class Transactions(Base):
    __tablename__ = 'transactions'
    __table_args__ = {'schema': SCHEMA_NAME}
    id = Column(Integer, primary_key=True, autoincrement=True)
    invoice_id = Column(String(50), nullable=True)
    line_number = Column(Integer)
    customer_id = Column(Integer, ForeignKey(f'{SCHEMA_NAME}.customers.customer_id'))
    product_id = Column(Integer, ForeignKey(f'{SCHEMA_NAME}.products.product_id'))
    size = Column(String(10))
    color = Column(String(50), nullable=True)
    unit_price = Column(Float)
    quantity = Column(Integer, nullable=True)
    transaction_date = Column(DateTime)
    discount = Column(Float, nullable=True)
    line_total = Column(Float)
    store_id = Column(Integer, ForeignKey(f'{SCHEMA_NAME}.stores.store_id'))
    employee_id = Column(Integer, ForeignKey(f'{SCHEMA_NAME}.employees.employee_id'))
    currency = Column(String(10), nullable=True)
    currency_symbol = Column(String(5))
    sku = Column(String(100))
    transaction_type = Column(String(20))
    payment_method = Column(String(50))
    invoice_total = Column(Float)
    is_return = Column(Boolean)

# Data processing functions
def process_stores(spark_df):
    spark_df = spark_df.withColumn("country", when(col("Country").isNull(), lit("Unknown")).otherwise(col("Country"))) \
                      .withColumn("city", when(col("City").isNull(), lit("Unknown")).otherwise(col("City"))) \
                      .withColumn("latitude", col("Latitude").cast(FloatType())) \
                      .withColumn("longitude", col("Longitude").cast(FloatType()))
    
    return spark_df.select(
        col("Store ID").alias("store_id"),
        col("country"),
        col("city"),
        col("Store Name").alias("store_name"),
        col("Number of Employees").alias("number_of_employees"),
        col("ZIP Code").alias("zip_code"),
        col("latitude"),
        col("longitude")
    )

def process_employees(spark_df):
    spark_df = spark_df.withColumn("name", when(col("Name").isNull(), lit("Unknown")).otherwise(col("Name"))) \
                      .withColumn("position", when(col("Position").isNull(), lit("Unknown")).otherwise(col("Position")))
    
    return spark_df.select(
        col("Employee ID").alias("employee_id"),
        col("Store ID").alias("store_id"),
        col("name"),
        col("position")
    )

def process_customers(spark_df):
    spark_df = spark_df.withColumn("join_date", to_date(col("Date Of Birth"), "yyyy-MM-dd")) \
                      .withColumn("name", when(col("Name").isNull(), lit("Unknown")).otherwise(col("Name"))) \
                      .withColumn("email", when(col("Email").isNull(), lit("unknown@example.com")).otherwise(col("Email")))
    
    return spark_df.select(
        col("Customer ID").alias("customer_id"),
        col("name"),
        col("email"),
        col("join_date"),
        col("Telephone").alias("telephone"),
        col("City").alias("city"),
        col("Country").alias("country"),
        col("Gender").alias("gender"),
        to_date(col("Date Of Birth"), "yyyy-MM-dd").alias("date_of_birth"),
        col("Job Title").alias("job_title")
    )

def process_products(spark_df):
    spark_df = spark_df.withColumn("category", when(col("Category").isNull(), lit("Unknown")).otherwise(col("Category"))) \
                      .withColumn("color", when(col("Color").isNull(), lit("Unknown")).otherwise(col("Color"))) \
                      .withColumn("sizes", when(col("Sizes").isNull(), lit("Unknown")).otherwise(col("Sizes")))
    
    return spark_df.select(
        col("Product ID").alias("product_id"),
        col("category"),
        col("Sub Category").alias("sub_category"),
        col("Description PT").alias("description_pt"),
        col("Description DE").alias("description_de"),
        col("Description FR").alias("description_fr"),
        col("Description ES").alias("description_es"),
        col("Description EN").alias("description_en"),
        col("Description ZH").alias("description_zh"),
        col("color"),
        col("sizes"),
        col("Production Cost").alias("production_cost")
    )

def process_discounts(spark_df):
    spark_df = spark_df.withColumn("description", when(col("Description").isNull(), lit("No description")).otherwise(col("Description"))) \
                      .withColumn("category", when(col("Category").isNull(), lit("General")).otherwise(col("Category")))
    
    return spark_df.select(
        to_date(col("Start"), "yyyy-MM-dd").alias("start_date"),
        to_date(col("End"), "yyyy-MM-dd").alias("end_date"),
        col("Discont").alias("discount_rate"),
        col("description"),
        col("category"),
        col("Sub Category").alias("sub_category")
    )

def process_transactions(spark_df):
    spark_df = spark_df.withColumn("invoice_id", when(col("Invoice ID").isNull(), lit("UNKNOWN")).otherwise(col("Invoice ID"))) \
                      .withColumn("quantity", when(col("Quantity").isNull(), lit(1)).otherwise(col("Quantity"))) \
                      .withColumn("color", when(col("Color").isNull(), lit("Unknown")).otherwise(col("Color"))) \
                      .withColumn("discount", when(col("Discount").isNull(), lit(0.0)).otherwise(col("Discount"))) \
                      .withColumn("currency", when(col("Currency").isNull(), lit("USD")).otherwise(col("Currency"))) \
                      .withColumn("is_return", col("Transaction Type") == "Return")
    
    return spark_df.select(
        col("invoice_id"),
        col("Line").alias("line_number"),
        col("Customer ID").alias("customer_id"),
        col("Product ID").alias("product_id"),
        col("Size").alias("size"),
        col("color"),
        col("Unit Price").alias("unit_price"),
        col("quantity"),
        col("Date").alias("transaction_date"),
        col("discount"),
        col("Line Total").alias("line_total"),
        col("Store ID").alias("store_id"),
        col("Employee ID").alias("employee_id"),
        col("currency"),
        col("Currency Symbol").alias("currency_symbol"),
        col("SKU").alias("sku"),
        col("Transaction Type").alias("transaction_type"),
        col("Payment Method").alias("payment_method"),
        col("Invoice Total").alias("invoice_total"),
        col("is_return")
    )

# In setup_database function, add table creation after schema creation
def setup_database(engine):
    """Create schema and set permissions if needed"""
    try:
        with engine.begin() as conn:
            # Check if schema exists
            schema_exists = conn.execute(
                text("SELECT 1 FROM information_schema.schemata WHERE schema_name = :schema"),
                {'schema': SCHEMA_NAME}
            ).scalar()
            
            if not schema_exists:
                logger.info(f"Creating schema {SCHEMA_NAME}")
                conn.execute(text(f"CREATE SCHEMA {SCHEMA_NAME}"))
                conn.execute(text(f"""
                    GRANT ALL ON SCHEMA {SCHEMA_NAME} TO {os.getenv('DB_USER', 'postgres')};
                    ALTER DEFAULT PRIVILEGES IN SCHEMA {SCHEMA_NAME} 
                    GRANT ALL ON TABLES TO {os.getenv('DB_USER', 'postgres')};
                """))
            
            # Create all tables if they don't exist
                Base.metadata.create_all(engine)
            
        return True
    except Exception as e:
        logger.error(f"Database setup failed: {str(e)}")
        return False

def upsert_data(session, table_class, records, primary_key, batch_size=1000):
    """Upsert data into the database with batch processing"""
    try:
        total_records = len(records)
        logger.info(f"Starting to upsert {total_records} records in batches of {batch_size}")
        
        for i in range(0, total_records, batch_size):
            batch = records[i:i + batch_size]
            batch_num = (i // batch_size) + 1
            total_batches = (total_records - 1) // batch_size + 1
            
            try:
                session.bulk_insert_mappings(table_class, batch)
                session.commit()
                logger.info(f"Successfully inserted batch {batch_num}/{total_batches}")
            except Exception as e:
                session.rollback()
                logger.info(f"Insert failed for batch {batch_num}, switching to upsert mode")
                
                for record in batch:
                    try:
                        existing = session.query(table_class).filter(
                            getattr(table_class, primary_key) == record[primary_key]
                        ).first()
                        
                        if existing:
                            for key, value in record.items():
                                setattr(existing, key, value)
                        else:
                            new_record = table_class(**record)
                            session.add(new_record)
                    except Exception as rec_error:
                        logger.error(f"Error processing record {record[primary_key]}: {str(rec_error)}")
                        continue
                
                try:
                    session.commit()
                    logger.info(f"Successfully upserted batch {batch_num}/{total_batches}")
                except Exception as commit_error:
                    session.rollback()
                    logger.error(f"Failed to commit batch {batch_num}: {str(commit_error)}")
                    continue
                
    except Exception as e:
        session.rollback()
        logger.error(f"Error in upsert_data: {str(e)}")
        raise

def write_spark_df_to_postgres(spark_df, table_name, engine):
    """Write Spark DataFrame to PostgreSQL with optimized methods"""
    try:
        table_class = globals()[table_name.capitalize()]
        primary_key = {
            'stores': 'store_id',
            'employees': 'employee_id',
            'customers': 'customer_id',
            'products': 'product_id',
            'discounts': 'id',
            'transactions': 'id'
        }.get(table_name)
        
        if not primary_key:
            logger.error(f"Primary key not defined for table {table_name}")
            return
            
        # Skip count verification for large tables to avoid Spark worker crashes
        if table_name in ['transactions']:  # Add other large tables if needed
            total_rows = "unknown (skipped count for large table)"
        else:
            total_rows = spark_df.count()
            
        logger.info(f"Processing {total_rows} records for table {table_name}")
        
        # For small tables, use direct conversion
        if isinstance(total_rows, int) and total_rows <= 10000:
            logger.info(f"Processing small {table_name} table with direct conversion")
            pandas_df = spark_df.toPandas()
            records = pandas_df.to_dict('records')
            
            Session = sessionmaker(bind=engine)
            session = Session()
            try:
                upsert_data(session, table_class, records, primary_key)
                
                # Verify count using SQLAlchemy which handles schema names correctly
                count = session.query(table_class).count()
                logger.info(f"Verified {count} records in {table_name}")
                
            finally:
                session.close()
            return
        
        # For larger tables
        logger.info(f"Processing large {table_name} table with optimized chunks")
        pandas_iter = spark_df.toLocalIterator() 
        records_buffer = []
        
        for i, row in enumerate(pandas_iter, 1):
            records_buffer.append(row.asDict())
            
            if i % 10000 == 0 or (isinstance(total_rows, int) and i == total_rows):
                Session = sessionmaker(bind=engine)
                session = Session()
                try:
                    upsert_data(session, table_class, records_buffer, primary_key)
                    logger.info(f"Processed {i} records")
                    records_buffer = []
                finally:
                    session.close()
        
        # Skip final verification for large tables
        if table_name not in ['transactions']:  # Add other large tables if needed
            with engine.connect() as conn:
                # Use quoted identifiers for the schema and table name
                count = conn.execute(text(f'SELECT COUNT(*) FROM "{SCHEMA_NAME}"."{table_name}"')).scalar()
                logger.info(f"Final count for {table_name}: {count} records")
                
    except Exception as e:
        logger.error(f"Error writing to {table_name}: {str(e)}", exc_info=True)
        raise

def load_data_to_postgres():
    """Main function to load data to PostgreSQL"""
    #DB_USER = os.getenv('DB_USER', 'postgres')
    #DB_PASSWORD = os.getenv('DB_PASSWORD', 'Grandvalley_2026')
    #DB_HOST = os.getenv('DB_HOST', 'localhost')
    #DB_PORT = os.getenv('DB_PORT', '5432')
    #DB_NAME = os.getenv('DB_NAME', 'Global_Fashion_Retail_Stores')
    #DATASET_PATH = os.getenv('DATASET_PATH', 'C:/Users/megin/fashion_dataset')
    DB_USER = os.getenv('DB_USER')
    DB_PASSWORD = os.getenv('DB_PASSWORD')
    DB_HOST = os.getenv('DB_HOST')
    DB_PORT = os.getenv('DB_PORT')
    DB_NAME = os.getenv('DB_NAME')
    DATASET_PATH = os.getenv('DATASET_PATH')

    required_vars = {
    'DB_USER': DB_USER,
    'DB_PASSWORD': DB_PASSWORD,
    'DB_HOST': DB_HOST,
    'DB_NAME': DB_NAME,
    'DATASET_PATH': DATASET_PATH
    }
    
    for var_name, var_value in required_vars.items():
        if not var_value:
            raise ValueError(f"Missing required environment variable: {var_name}")
    
    # Set default for PORT only (since it often doesn't change)
    DB_PORT = DB_PORT if DB_PORT else '5432'
    try:
        engine = create_engine(
            f'postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}',
            pool_size=20,
            max_overflow=30,
            pool_pre_ping=True,
            pool_recycle=3600,
            connect_args={'connect_timeout': 10}
        )
        
        if not setup_database(engine):
            raise RuntimeError("Database setup failed")
        
        Base.metadata.create_all(engine)
        
        file_processors = [
            ('stores.csv', process_stores, 'stores'),
            ('employees.csv', process_employees, 'employees'),
            ('products.csv', process_products, 'products'),
            ('discounts.csv', process_discounts, 'discounts'),
            ('customers.csv', process_customers, 'customers'),
            ('transactions.csv', process_transactions, 'transactions')
        ]
        
        for file_name, processor, table_name in file_processors:
            file_path = os.path.join(DATASET_PATH, file_name)
            if os.path.exists(file_path):
                logger.info(f"Processing {file_name}...")
                start_time = datetime.now()
                
                try:
                    if file_name in ['customers.csv', 'transactions.csv']:
                        sample_df = spark.read.csv(file_path, header=True, inferSchema=True, samplingRatio=0.1)
                        schema = sample_df.schema
                        spark_df = spark.read.csv(file_path, header=True, schema=schema)
                    else:
                        spark_df = spark.read.csv(file_path, header=True, inferSchema=True)
                    
                    processed_df = processor(spark_df)
                    logger.info(f"Processed DataFrame schema: {processed_df._jdf.schema().treeString()}")
                    
                    write_spark_df_to_postgres(processed_df, table_name, engine)
                    
                    logger.info(f"Completed {file_name} in {datetime.now() - start_time}")
                except Exception as e:
                    logger.error(f"Error processing {file_name}: {str(e)}", exc_info=True)
                    
                    if file_name == 'products.csv':
                        logger.info("Attempting alternative approach for products.csv")
                        try:
                            products_pd = pd.read_csv(file_path)
                            products_spark = spark.createDataFrame(products_pd)
                            processed_df = processor(products_spark)
                            write_spark_df_to_postgres(processed_df, table_name, engine)
                            logger.info(f"Successfully processed products.csv with alternative approach")
                        except Exception as alt_e:
                            logger.error(f"Alternative approach also failed: {str(alt_e)}")
                            raise
                    else:
                        raise
            else:
                logger.warning(f"File not found: {file_path}")
        
        logger.info("Data load completed successfully")
        
    except Exception as e:
        logger.error(f"Database error: {str(e)}", exc_info=True)
        raise
    finally:
        engine.dispose()
        spark.stop()

if __name__ == "__main__":
    try:
        logger.info("Starting data load process...")
        start_time = datetime.now()
        load_data_to_postgres()
        logger.info(f"Process completed in {datetime.now() - start_time}")
    except Exception as e:
        logger.error(f"Fatal error: {str(e)}", exc_info=True)
        exit(1)

25/04/22 22:58:04 WARN Utils: Your hostname, BOOK-KTS1KMKSNJ resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/22 22:58:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/04/22 22:58:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/22 22:58:05 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


2025-04-22 22:58:06,937 - INFO - Starting data load process...


2025-04-22 22:58:06,939 - ERROR - Fatal error: Missing required environment variable: DB_USER
Traceback (most recent call last):
  File "/tmp/ipykernel_19999/3184093236.py", line 531, in <module>
    load_data_to_postgres()
  File "/tmp/ipykernel_19999/3184093236.py", line 451, in load_data_to_postgres
    raise ValueError(f"Missing required environment variable: {var_name}")
ValueError: Missing required environment variable: DB_USER
